# Demo Técnica: Reportes Ejecutivos Automáticos con Vertex AI

**Arquitecto:** Google Cloud Senior Solutions  
**Objetivo:** Mostrar cómo una plataforma SaaS puede generar insights automáticos usando Gemini 1.5 Flash en Vertex AI

## Flujo de la Demo
```
Datos Multi-fuente → JSON → Gemini 1.5 Flash → Resumen Ejecutivo (Markdown)
```

## Por qué Vertex AI SDK (vs API directa)
- ✅ Autenticación IAM (más segura)
- ✅ Logging automático en Cloud Logging
- ✅ Quotas empresariales más altas
- ✅ Ideal para producción en SaaS

---

## 📋 Prerequisitos (Ejecutar antes de la demo)

```bash
# 1. Instalar dependencias
pip install google-cloud-aiplatform

# 2. Autenticación con Google Cloud
gcloud auth application-default login

# 3. Habilitar API de Vertex AI
gcloud services enable aiplatform.googleapis.com
```

## 1️⃣ Imports y Librerías

Usamos el **SDK oficial de Vertex AI** (`google-cloud-aiplatform`), no la API directa de Gemini.

In [30]:
import json
import os
from dotenv import load_dotenv
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, GenerationConfig

# Cargar variables de entorno desde .env
load_dotenv()

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 2️⃣ Configuración de Vertex AI

**IMPORTANTE:** Reemplaza `PROJECT_ID` con tu Project ID real de Google Cloud.

In [31]:
# ============================================================================
# CONFIGURACIÓN INICIAL DE VERTEX AI
# ============================================================================
# IMPORTANTE: Vertex AI usa autenticación IAM (NO API key)
# La autenticación se hace con: gcloud auth application-default login

# Cargar configuración desde .env (buena práctica de seguridad)
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "prj-uc-lab-liseth")
LOCATION = os.getenv("GCP_LOCATION", "us-central1")

# Inicializar Vertex AI SDK (necesario antes de usar cualquier modelo)
try:
    aiplatform.init(project=PROJECT_ID, location=LOCATION)
    print(f"✅ Vertex AI inicializado correctamente")
    print(f"   - Proyecto: {PROJECT_ID}")
    print(f"   - Región: {LOCATION}")
    print(f"\n📌 Nota: Vertex AI usa autenticación IAM, NO API key")
except Exception as e:
    print(f"❌ Error al inicializar Vertex AI: {str(e)}")
    print("\n🔧 Soluciones:")
    print("1. Ejecuta: gcloud auth application-default login")
    print("2. Verifica que PROJECT_ID sea correcto")
    print("3. Verifica que Vertex AI API esté habilitada")

✅ Vertex AI inicializado correctamente
   - Proyecto: prj-uc-lab-liseth
   - Región: us-central1

📌 Nota: Vertex AI usa autenticación IAM, NO API key


## 3️⃣ Datos Simulados (Mock Data)

Simulamos datos consolidados de múltiples fuentes:
- **Google Ads**: Métricas de campañas publicitarias
- **Meta Ads**: Facebook/Instagram
- **Salesforce CRM**: Pipeline de ventas y conversiones
- **Shopify**: E-commerce y transacciones

En producción, estos datos vendrían de tus APIs o Data Warehouse.

In [32]:
# ============================================================================
# DATOS SIMULADOS (MOCK DATA)
# ============================================================================
saas_consolidated_data = {
    "cliente": "TechStartup S.A.S",
    "periodo": "Q1 2024",
    "fecha_reporte": "2024-03-31",
    "canales": {
        "google_ads": {
            "plataforma": "Google Ads",
            "metricas": {
                "gasto": 15000.00,  # USD
                "conversiones": 450,
                "ingresos": 67500.00,
                "impresiones": 1250000,
                "clicks": 18750,
                "ctr": 1.5,  # Click-Through Rate (%)
                "cpc": 0.80,  # Costo Por Click (USD)
                "conversion_rate": 2.4,  # %
                "roas": 4.5  # Return On Ad Spend (ingresos/gasto)
            },
            "campanas_top": [
                {"nombre": "Search Brand", "roas": 8.2, "gasto": 5000},
                {"nombre": "Shopping Generic", "roas": 3.1, "gasto": 7000},
                {"nombre": "Display Remarketing", "roas": 2.8, "gasto": 3000}
            ]
        },
        "meta_ads": {
            "plataforma": "Meta Ads (Facebook/Instagram)",
            "metricas": {
                "gasto": 12000.00,
                "conversiones": 280,
                "ingresos": 33600.00,
                "impresiones": 2500000,
                "clicks": 25000,
                "ctr": 1.0,
                "cpc": 0.48,
                "conversion_rate": 1.12,
                "roas": 2.8  # Menor ROAS que Google Ads
            },
            "campanas_top": [
                {"nombre": "FB Prospecting Lookalike", "roas": 3.5, "gasto": 6000},
                {"nombre": "IG Stories Engagement", "roas": 2.2, "gasto": 4000},
                {"nombre": "FB Carousel Retargeting", "roas": 2.5, "gasto": 2000}
            ]
        },
        "salesforce": {
            "plataforma": "Salesforce CRM",
            "metricas": {
                "leads_generados": 730,  # Total de leads del trimestre
                "leads_calificados_marketing": 520,  # MQL
                "leads_calificados_ventas": 285,  # SQL
                "oportunidades_creadas": 180,
                "deals_cerrados_ganados": 95,
                "ingresos_pipeline": 450000.00,  # Ingresos proyectados en pipeline
                "ingresos_cerrados": 285000.00,  # Ingresos reales cerrados
                "tasa_conversion_lead_to_sql": 39.0,  # % (285/730)
                "tasa_conversion_sql_to_deal": 52.8,  # % (95/180)
                "ciclo_venta_promedio_dias": 45,
                "ticket_promedio": 3000.00  # USD por deal
            },
            "fuentes_lead": {
                "google_ads": 450,  # Leads atribuidos a Google Ads
                "meta_ads": 280,  # Leads atribuidos a Meta Ads
                "organico": 0  # Sin tráfico orgánico significativo
            }
        },
        "shopify": {
            "plataforma": "Shopify E-commerce",
            "metricas": {
                "pedidos_totales": 730,  # Coincide con conversiones totales ads
                "ingresos_totales": 101100.00,  # USD (suma de ingresos de ads)
                "ticket_promedio_ecommerce": 138.49,  # USD
                "tasa_abandono_carrito": 68.5,  # %
                "sesiones_tienda": 125000,
                "tasa_conversion_tienda": 0.58,  # % (730/125000)
                "categorias_top": [
                    {"nombre": "Software Licenses", "ingresos": 45500, "unidades": 325},
                    {"nombre": "Consultancy Hours", "ingresos": 35600, "unidades": 178},
                    {"nombre": "Training Courses", "ingresos": 20000, "unidades": 227}
                ],
                "metodos_pago": {
                    "tarjeta_credito": 65,  # %
                    "paypal": 25,
                    "transferencia": 10
                }
            }
        }
    },
    "kpis_consolidados": {
        "inversion_total_marketing": 27000.00,  # Google + Meta
        "ingresos_atribuidos_marketing": 101100.00,  # Desde Shopify
        "roas_consolidado": 3.74,  # (101100 / 27000)
        "cac": 36.99,  # Customer Acquisition Cost (27000 / 730)
        "ltv_estimado": 9000.00,  # Lifetime Value estimado
        "ratio_ltv_cac": 243.3,  # Excelente ratio (>3:1 es saludable)
        "roi_marketing": 274.4  # % ((101100-27000)/27000 * 100)
    },
    "alertas_automaticas": [
        "Meta Ads tiene un ROAS 38% menor que Google Ads (2.8 vs 4.5)",
        "Tasa de abandono de carrito del 68.5% está por encima del benchmark (65%)",
        "Solo el 39% de leads se convierten en SQL - posible problema de calidad de tráfico"
    ]
}

print("✅ Datos simulados cargados")
print(f"   - Cliente: {saas_consolidated_data['cliente']}")
print(f"   - Período: {saas_consolidated_data['periodo']}")
print(f"   - Canales integrados: {len(saas_consolidated_data['canales'])}")

✅ Datos simulados cargados
   - Cliente: TechStartup S.A.S
   - Período: Q1 2024
   - Canales integrados: 4


## 4️⃣ Configuración del Modelo Gemini

### Por qué Gemini 1.5 Flash:
1. **Menor latencia:** ~2-3 segundos vs 5-8 segundos de Pro
2. **Menor costo:** ~70% más económico que Pro
3. **Suficiente capacidad:** Perfecto para análisis de JSON estructurado

### Configuración de `temperature=0.2`:
- Reduce "alucinaciones" en números
- Mantiene consistencia en respuestas
- Ideal para reportes ejecutivos

In [44]:
TEMPERATURE = 0.2
MAX_OUTPUT_TOKENS = 16500
MODEL_NAME = os.getenv("GEMINI_MODEL", "gemini-2.5-flash")

generation_config = {
    "temperature": TEMPERATURE,
    "max_output_tokens": MAX_OUTPUT_TOKENS
}

print("✅ Configuración del modelo establecida")
print(f"   - Modelo: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE} (baja = menos alucinaciones)")
print(f"   - Max tokens: {MAX_OUTPUT_TOKENS}")


✅ Configuración del modelo establecida
   - Modelo: gemini-2.5-flash
   - Temperature: 0.2 (baja = menos alucinaciones)
   - Max tokens: 16500


## 🧪 Test de Conectividad (Opcional - ejecuta para verificar que todo funciona)

Ejecuta esta celda para hacer un test rápido con el modelo antes de la demo completa.

## 5️⃣ Función Principal: Generación de Resumen Inteligente

### Arquitectura de la función:
1. **Serializa datos a JSON** (formato óptimo para LLMs)
2. **Construye prompt estructurado** con técnica de "Role Prompting"
3. **Llama a Gemini vía Vertex AI SDK** (usa IAM, no API key)
4. **Retorna Markdown** con insights accionables

In [45]:
# ============================================================================
# FUNCIÓN PRINCIPAL: GENERACIÓN DE RESUMEN INTELIGENTE
# ============================================================================
def generate_smart_summary(data: dict) -> str:
    """
    Genera un resumen ejecutivo automatizado usando Gemini 1.5 Flash en Vertex AI.

    Args:
        data (dict): Diccionario con datos consolidados de marketing/ventas

    Returns:
        str: Resumen ejecutivo en formato Markdown con insights accionables
    """

    # Paso 1: Convertir datos Python a JSON string (mejor para LLMs que dict)
    # indent=2 hace el JSON legible para el modelo
    data_json = json.dumps(data, indent=2, ensure_ascii=False)

    # Paso 2: Construir el prompt con técnica de "Role Prompting"
    # Estructura: Rol + Contexto + Tarea + Formato + Restricciones
    prompt = f"""
Actúa como un experto en Growth Marketing con 10+ años de experiencia en análisis de datos.

**CONTEXTO:**
Tienes acceso a datos consolidados de un cliente SaaS que invierte en marketing digital
(Google Ads, Meta Ads) y usa Salesforce + Shopify para gestionar ventas.

**DATOS JSON:**
```json
{data_json}
```

**TAREA:**
Analiza los datos JSON y realiza lo siguiente:
1. Compara el rendimiento entre canales (Google Ads vs Meta Ads)
2. Detecta ineficiencias operativas o áreas de mejora
3. Identifica oportunidades de crecimiento o optimización

**FORMATO DE SALIDA:**
Genera un resumen ejecutivo con exactamente 3 puntos clave, usando:
- Formato Markdown (## para títulos, ** para negritas)
- Un emoji relevante al inicio de cada punto
- Datos numéricos concretos (incluye %, $, ROAS, etc.)
- Máximo 2 oraciones por punto
- Tono profesional pero accesible

**RESTRICCIONES:**
- NO inventes números que no están en los datos
- NO hagas recomendaciones genéricas sin fundamento en datos
- SI un número parece contradictorio, menciónalo

Genera el resumen ahora:
"""

    # Paso 3: Inicializar el modelo Gemini desde Vertex AI
    # GenerativeModel es la clase principal del SDK de Vertex AI
    model = GenerativeModel(
        model_name=MODEL_NAME,
        generation_config=generation_config
    )

    # Paso 4: Generar contenido (llamada al modelo)
    print("🤖 Generando resumen con Gemini 1.5 Flash en Vertex AI...")
    print("-" * 70)

    response = model.generate_content(prompt)

    # Paso 5: Extraer el texto generado con debugging
    print("\n🔍 DEBUG - Inspeccionando respuesta del modelo:")
    print(f"   - Tipo de respuesta: {type(response)}")
    print(f"   - Tiene atributo 'text': {hasattr(response, 'text')}")
    print(f"   - Tiene atributo 'candidates': {hasattr(response, 'candidates')}")
    
    # Intentar diferentes formas de extraer el contenido
    summary = ""
    
    try:
        # Método 1: Usar response.text directamente
        summary = response.text
        print(f"   - Longitud del texto: {len(summary)} caracteres")
        
        if not summary or len(summary) == 0:
            print("   ⚠️ WARNING: response.text está vacío")
            
            # Método 2: Intentar acceder a candidates
            if hasattr(response, 'candidates') and response.candidates:
                print(f"   - Número de candidates: {len(response.candidates)}")
                candidate = response.candidates[0]
                if hasattr(candidate, 'content'):
                    if hasattr(candidate.content, 'parts'):
                        summary = "".join([part.text for part in candidate.content.parts if hasattr(part, 'text')])
                        print(f"   - Texto extraído de candidates: {len(summary)} caracteres")
    
    except Exception as e:
        print(f"   ❌ Error al extraer texto: {str(e)}")
        print(f"   - Response completo: {response}")
        
    return summary

print("✅ Función generate_smart_summary() definida (con debugging)")

✅ Función generate_smart_summary() definida (con debugging)


## 6️⃣ Preview de los Datos (Opcional)

Muestra un resumen de los datos consolidados antes de enviarlos a la IA.

In [46]:
print("="*70)
print("DATOS CONSOLIDADOS DEL CLIENTE")
print("="*70 + "\n")

# Información general
print(f"📊 Cliente: {saas_consolidated_data['cliente']}")
print(f"📅 Período: {saas_consolidated_data['periodo']}")
print(f"💰 Inversión total marketing: ${saas_consolidated_data['kpis_consolidados']['inversion_total_marketing']:,.2f}")
print(f"💵 Ingresos atribuidos: ${saas_consolidated_data['kpis_consolidados']['ingresos_atribuidos_marketing']:,.2f}")
print(f"📈 ROAS consolidado: {saas_consolidated_data['kpis_consolidados']['roas_consolidado']}x")
print(f"🎯 CAC (Costo de Adquisición): ${saas_consolidated_data['kpis_consolidados']['cac']}")

# Comparación de canales
print("\n" + "-"*70)
print("COMPARACIÓN DE CANALES")
print("-"*70)
print(f"Google Ads  → Gasto: ${saas_consolidated_data['canales']['google_ads']['metricas']['gasto']:,.2f} | ROAS: {saas_consolidated_data['canales']['google_ads']['metricas']['roas']}x")
print(f"Meta Ads    → Gasto: ${saas_consolidated_data['canales']['meta_ads']['metricas']['gasto']:,.2f} | ROAS: {saas_consolidated_data['canales']['meta_ads']['metricas']['roas']}x")

# Alertas
print("\n" + "-"*70)
print("ALERTAS AUTOMÁTICAS DETECTADAS")
print("-"*70)
for i, alerta in enumerate(saas_consolidated_data['alertas_automaticas'], 1):
    print(f"{i}. ⚠️  {alerta}")

print("\n" + "="*70 + "\n")

DATOS CONSOLIDADOS DEL CLIENTE

📊 Cliente: TechStartup S.A.S
📅 Período: Q1 2024
💰 Inversión total marketing: $27,000.00
💵 Ingresos atribuidos: $101,100.00
📈 ROAS consolidado: 3.74x
🎯 CAC (Costo de Adquisición): $36.99

----------------------------------------------------------------------
COMPARACIÓN DE CANALES
----------------------------------------------------------------------
Google Ads  → Gasto: $15,000.00 | ROAS: 4.5x
Meta Ads    → Gasto: $12,000.00 | ROAS: 2.8x

----------------------------------------------------------------------
ALERTAS AUTOMÁTICAS DETECTADAS
----------------------------------------------------------------------
1. ⚠️  Meta Ads tiene un ROAS 38% menor que Google Ads (2.8 vs 4.5)
2. ⚠️  Tasa de abandono de carrito del 68.5% está por encima del benchmark (65%)
3. ⚠️  Solo el 39% de leads se convierten en SQL - posible problema de calidad de tráfico




## 7️⃣ Ejecutar Demo: Generar Resumen Ejecutivo

Esta es la celda principal de la demo. Ejecuta el análisis con IA.

In [47]:
# ============================================================================
# EJECUCIÓN PRINCIPAL DE LA DEMO
# ============================================================================

try:
    # Llamar a la función principal
    executive_summary = generate_smart_summary(saas_consolidated_data)

    # Mostrar resultado con formato
    print("\n" + "="*70)
    print("RESUMEN EJECUTIVO GENERADO POR IA")
    print("="*70 + "\n")
    
    if executive_summary and len(executive_summary) > 0:
        print(executive_summary)
    else:
        print("⚠️ ADVERTENCIA: El modelo no generó contenido.")
        print("\nPosibles causas:")
        print("1. Safety filters bloquearon la respuesta")
        print("2. Problema de configuración del modelo")
        print("3. Límite de tokens excedido")
        print("\n💡 Solución: Revisa el output de DEBUG arriba para más detalles")
    
    print("\n" + "="*70)
    print("✅ Demo completada exitosamente")
    print("="*70)

except Exception as e:
    print(f"\n❌ Error durante la ejecución: {str(e)}")
    print(f"\nTipo de error: {type(e).__name__}")
    print(f"\n🔍 Verifica:")
    print("1. Que hayas configurado PROJECT_ID correctamente")
    print("2. Que tengas credenciales de GCP configuradas (gcloud auth)")
    print("3. Que Vertex AI API esté habilitada en tu proyecto")
    print("4. Que tengas la librería instalada: pip install google-cloud-aiplatform")
    
    # Mostrar el traceback completo para debugging
    import traceback
    print("\n📋 Traceback completo:")
    traceback.print_exc()

c:\Users\memer\OneDrive\Documentos\Eme\Ucloud\proyectos\LUHN\venv\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


🤖 Generando resumen con Gemini 1.5 Flash en Vertex AI...
----------------------------------------------------------------------

🔍 DEBUG - Inspeccionando respuesta del modelo:
   - Tipo de respuesta: <class 'vertexai.generative_models._generative_models.GenerationResponse'>
   - Tiene atributo 'text': True
   - Tiene atributo 'candidates': True
   - Longitud del texto: 1146 caracteres

RESUMEN EJECUTIVO GENERADO POR IA

Aquí tienes un resumen ejecutivo de los datos proporcionados:

## Resumen Ejecutivo: Análisis de Crecimiento Q1 2024

📊 **Disparidad en Rendimiento de Canales:** Google Ads supera significativamente a Meta Ads con un ROAS de **4.5** frente a **2.8**, generando un 38% más de retorno por inversión. Se recomienda reevaluar la asignación de presupuesto, priorizando Google Ads y optimizando las campañas de Meta Ads con bajo rendimiento.

📉 **Fugas Críticas en el Embudo de Conversión:** Se identifican dos fugas importantes: una tasa de abandono de carrito del **68.5%** en Sho

---

## 📝 Notas Finales

### Costos estimados de esta demo:
- **Input:** ~4,500 caracteres × $0.075/1M = $0.00034
- **Output:** ~500 caracteres × $0.30/1M = $0.00015
- **Total:** ~$0.0005 por ejecución (medio centavo)

### Para 10,000 reportes mensuales:
- **Costo mensual:** ~$5 USD 🚀
